In [10]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("hsankesara/flickr-image-dataset")

print("Path to dataset files:", path)

Download already complete (8765396518 bytes).
Extracting files...


OSError: [Errno 28] No space left on device

In [ ]:
import numpy as np
from PIL import Image
import os
import string
from pickle import dump
from pickle import load
from keras.applications.xception import Xception
from keras.applications.xception import pre_process_input
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.text import Tokenizer #for text tokenization
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers.merge import add
from keras.models import Model, load_model
from keras.layers import Input, Dense #Keras to build our CNN and LSTM
from keras.layers import LSTM, Embedding, Dropout
from tqdm import tqdm_notebook as tqdm #to check loop progress
tqdm().pandas()

DATA CLEANING

In [ ]:
# create a dictionary that maps images with all 5 captions
def mapImgWithCaptions(filename):
    file = open(filename, "r")
    text = file.read()
    captions = text.split('n')
    descriptions = {}
    for caption in captions[:-1]:
        img, caption = caption.split('t')
        if img[:-2] not in descriptions:
            descriptions[img[:-2]] = [caption]
        else:
            descriptions[img[:-2]].append(caption)
    return descriptions


# clean the data by taking all descriptions as input. 
# uppercase to lowercase conversion, punctuation removal, and removal of the number containing words.
def txt_clean(captions):
    table = str.maketrans('', '', string.punctuation)
    for img, caps in captions.items():
        for i, img_caption in enumerate(caps):
            img_caption.replace("-", " ")
            descp = img_caption.split()
            # uppercase to lowercase
            descp = [wrd.lower() for wrd in descp]
            # remove punctuation from each token
            descp = [wrd.translate(table) for wrd in descp]
            # remove hanging 's and a
            descp = [wrd for wrd in descp if (len(wrd) > 1)]
            # remove words containing number with them
            descp = [wrd for wrd in descp if (wrd.isalpha())]
            # convert back to strings
            img_caption = ' '.join(descp)
            captions[img][i] = img_caption
    return captions

dataset_text = " - Image Caption GeneratorFlickr_8k_text"
dataset_images = " - Image Caption GeneratorFlickr_8k_Datatset"
filename = dataset_text + "/" + "Flickr8k.token.txt"
descriptions = mapImgWithCaptions(filename)

In [ ]:
# extract the features from the model
model = Xception(include_top=False, pooling='avg')